# CNN
## Convolutional Neural Network　畳込みニューラルネットワーク

### (1) 入力層
28x28=784次元のベクトルが入力ベクトルです。<br>
数字の画像を784ピクセルに分割し、1次元=1ピクセルで情報を代入しています。

### (2) 畳込み層1 -> プーリング層1
2層準備。ゼロパディング(zero padding)という補完機能を利用しています。<br>
ファイルの畳込みに対して、周縁データが欠落するのですが、その補完のためにパディングを用います。<br>
（パディングの話で推測できるように、畳込み層では、入力の次元圧縮を行いません）<br>
プーリング(pooling)では、畳込みで返された数値を圧縮します。<br>
ここでは、最大プーリング(max pooling)という、最も基本的な手法を用いています。<br>
プーリングでも、パディングを行うことがあります。<br>
（重ね合わせがずれる領域が存在することがあるため）

* ゼロパディング(zero padding)とは、画像や帳票などで数値を表現する際、<br>
書式で指定した桁数に満たない部分をゼロで埋めることである。<br>
例えば「123」を5桁で表す場合、2桁足りない部分をゼロで埋めて「00123」と表記する<br>
* 最大プーリング(max pooling)とは、設定範囲のピクセルの中で最大値を選択する

### (3) 畳込み層2 -> プーリング層2
(2)の繰り返しでさらに圧縮 -> この段階で7x7=49次元まで落とす予定です。

### (4) 全結合層(高密度結合層)
抽出したプーリング層からの出力層を入力層に送り、NNのような処理をかけます。<br>
活性化関数はReLUです。

### (5) クラス分類処理
ソフトマックス関数の計算 -> 交差エントロピー誤差関数による評価

https://qiita.com/icoxfog417/items/5fd55fad152231d706c2



In [1]:
import tensorflow as tf
import input_data
import numpy as np
import time
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
np.set_printoptions(suppress=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
# Network Parameters
n_input = 784  # MNIST data input (img shape: 28*28)
n_classes = 10  # MNIST total classes (0-9 digits)
train_dropout = 0.8  # Dropout for train, probability to keep units
test_dropout = 1.0  # Dropout for test, probability to keep units

# Training Parameters
learning_rate = 0.001
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)
training_epochs = 10

# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y_true = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)  # dropout (keep probability)

# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7 * 7 * 64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}
biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [3]:
# Create model


def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(
        x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')


def conv_net(x, weights, biases, dropout):
    # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
    # Reshape to match picture format [Height x Width x Channel]
    # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [4]:
# Construct model
y_pred = conv_net(x, weights, biases, keep_prob)
y_softmax = tf.nn.softmax(y_pred)

# Define loss and optimizer
# type 1:
# loss = tf.reduce_mean(
#     -tf.reduce_sum(y_true * tf.log(y_softmax), reduction_indices=[1]))
# type 2:
# loss = tf.reduce_mean(
#     tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred))
# type 3:
loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_true, logits=y_pred))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

# Evaluate model
pred = tf.argmax(y_pred, 1)
true = tf.argmax(y_true, 1)
correct_prediction = tf.equal(pred, true)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [5]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

# Training cycle
start = time.time()
for epoch_i in range(training_epochs):
    ave_cost = 0
    for batch_i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        _, c = sess.run(
            [optimizer, loss],
            feed_dict={
                x: batch_xs,
                y_true: batch_ys,
                keep_prob: train_dropout
            })
        ave_cost += c / total_batch
    # Display logs per epoch step
    if epoch_i % 1 == 0:
        train_acc = accuracy.eval(feed_dict={
            x: batch_xs,
            y_true: batch_ys,
            keep_prob: train_dropout
        })
        test_acc = accuracy.eval(feed_dict={
            x: mnist.test.images,
            y_true: mnist.test.labels,
            keep_prob: test_dropout
        })
        print("Epoch:%3d Batch:%5d " % (epoch_i,
                                        batch_i), "train_acc=%.5f" % train_acc,
              "test_acc=%.5f" % test_acc, "train_cost=%.9f" % ave_cost)
end = time.time()
print("Process Time :%.2f s" % (end - start))

# Calculate accuracy
print(
    sess.run(
        accuracy,
        feed_dict={
            x: mnist.test.images,
            y_true: mnist.test.labels,
            keep_prob: test_dropout
        }))
sess.close()

Epoch:  0 Batch:  549  train_acc=0.92000 test_acc=0.94910 train_cost=5249.972303328
Epoch:  1 Batch:  549  train_acc=0.93000 test_acc=0.96660 train_cost=709.464661671
Epoch:  2 Batch:  549  train_acc=0.97000 test_acc=0.97000 train_cost=368.078497117
Epoch:  3 Batch:  549  train_acc=0.95000 test_acc=0.97150 train_cost=221.170068842
Epoch:  4 Batch:  549  train_acc=0.98000 test_acc=0.97550 train_cost=157.183320430
Epoch:  5 Batch:  549  train_acc=0.98000 test_acc=0.97570 train_cost=112.156512499
Epoch:  6 Batch:  549  train_acc=0.99000 test_acc=0.97720 train_cost=82.397679098
Epoch:  7 Batch:  549  train_acc=0.98000 test_acc=0.97580 train_cost=65.240254770
Epoch:  8 Batch:  549  train_acc=0.99000 test_acc=0.98150 train_cost=55.077338686
Epoch:  9 Batch:  549  train_acc=0.97000 test_acc=0.97500 train_cost=41.081591611
Process Time :646.61 s
0.975
